# Magentic Example

In [11]:
import os
from dotenv import load_dotenv
from magentic.chat_model.openai_chat_model import OpenaiChatModel

load_dotenv()

chat_model = OpenaiChatModel(
    api_key=os.getenv('OPENAI_API_KEY'),
    base_url=os.getenv('OPENAI_API_ENDPOINT'),
    api_type="openai",
    model="gpt-4o",
)

In [14]:
# https://magentic.dev/logging-and-tracing/
import logfire

logfire.configure(send_to_logfire="if-token-present")
logfire.instrument_openai()
logfire.info("Hello, world!")

13:04:53.929 Hello, world!


## @prompt

In [15]:
from magentic import prompt


@prompt('Add more "dude"ness to: {phrase}', model=chat_model)
def dudeify(phrase: str) -> str: ...  # No function body as this is never executed


dudeify("Hello, how are you?")

13:05:04.594 Calling prompt-function dudeify
13:05:04.597   Chat Completion with 'gpt-4o' [LLM]
13:05:06.001   streaming response from 'gpt-4o' took 0.18s [LLM]


"Hey dude, how's it going?"

In [16]:
from magentic import prompt
from pydantic import BaseModel


class Superhero(BaseModel):
    name: str
    age: int
    power: str
    enemies: list[str]


@prompt("Create a Superhero named {name}.", model=chat_model) 
def create_superhero(name: str) -> Superhero: ...


create_superhero("Garden Man")
# Superhero(name='Garden Man', age=30, power='Control over plants', enemies=['Pollution Man', 'Concrete Woman'])

13:05:52.680 Calling prompt-function create_superhero
13:05:52.686   Chat Completion with 'gpt-4o' [LLM]
13:05:55.317   streaming response from 'gpt-4o' took 0.26s [LLM]


Superhero(name='Garden Man', age=35, power='Control over plant life', enemies=['Weed Whacker', 'Deforestation Duo', 'Concrete Crusher'])

## @chatprompt

In [17]:
from magentic import chatprompt, AssistantMessage, SystemMessage, UserMessage
from pydantic import BaseModel


class Quote(BaseModel):
    quote: str
    character: str


@chatprompt(
    SystemMessage("You are a movie buff."),
    UserMessage("What is your favorite quote from Harry Potter?"),
    AssistantMessage(
        Quote(
            quote="It does not do to dwell on dreams and forget to live.",
            character="Albus Dumbledore",
        )
    ),
    UserMessage("What is your favorite quote from {movie}?"),
    model=chat_model,
)
def get_movie_quote(movie: str) -> Quote: ...


get_movie_quote("Iron Man")
# Quote(quote='I am Iron Man.', character='Tony Stark')

13:12:43.629 Calling chatprompt-function get_movie_quote
13:12:43.638   Chat Completion with 'gpt-4o' [LLM]
13:12:45.080   streaming response from 'gpt-4o' took 0.16s [LLM]


Quote(quote='I am Iron Man.', character='Tony Stark')

## FunctionCall

In [18]:
from typing import Literal

from magentic import prompt, FunctionCall


def search_twitter(query: str, category: Literal["latest", "people"]) -> str:
    """Searches Twitter for a query."""
    print(f"Searching Twitter for {query!r} in category {category!r}")
    return "<twitter results>"


def search_youtube(query: str, channel: str = "all") -> str:
    """Searches YouTube for a query."""
    print(f"Searching YouTube for {query!r} in channel {channel!r}")
    return "<youtube results>"


@prompt(
    "Use the appropriate search function to answer: {question}",
    functions=[search_twitter, search_youtube],
    model=chat_model,
)
def perform_search(question: str) -> FunctionCall[str]: ...


output = perform_search("What is the latest news on LLMs?")
print(output)
# > FunctionCall(<function search_twitter at 0x10c367d00>, 'LLMs', 'latest')
output()
# > Searching Twitter for 'Large Language Models news' in category 'latest'
# '<twitter results>'

13:15:45.765 Calling prompt-function perform_search
13:15:45.775   Chat Completion with 'gpt-4o' [LLM]
13:15:47.319   streaming response from 'gpt-4o' took 0.17s [LLM]
FunctionCall(<function search_twitter at 0x1100436a0>, 'latest news on LLMs', 'latest')
13:15:47.327 Executing function call search_twitter
Searching Twitter for 'latest news on LLMs' in category 'latest'


'<twitter results>'

## @prompt_chain

In [19]:
from magentic import prompt_chain


def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    # Pretend to query an API
    return {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }


@prompt_chain(
    "What's the weather like in {city}?",
    functions=[get_current_weather],
    model=chat_model,
)
def describe_weather(city: str) -> str: ...


describe_weather("Boston")
# 'The current weather in Boston is 72°F and it is sunny and windy.'

13:19:55.230 Calling prompt-chain describe_weather
13:19:55.238   Chat Completion with 'gpt-4o' [LLM]
13:19:56.590   streaming response from 'gpt-4o' took 0.04s [LLM]
13:19:56.590   Executing function call get_current_weather
13:19:56.597   Chat Completion with 'gpt-4o' [LLM]
13:19:57.477   streaming response from 'gpt-4o' took 0.24s [LLM]


'The weather in Boston is currently 72°F with sunny and windy conditions.'

## ParallelFunctionCall with @chatprompt

In [21]:
from magentic import (
    chatprompt,
    AssistantMessage,
    FunctionCall,
    FunctionResultMessage,
    ParallelFunctionCall,
    UserMessage,
)


def plus(a: int, b: int) -> int:
    return a + b


def minus(a: int, b: int) -> int:
    return a - b


plus_1_2 = FunctionCall(plus, 1, 2)
minus_2_1 = FunctionCall(minus, 2, 1)


@chatprompt(
    UserMessage(
        "Sum 1 and 2. Also subtract 1 from 2.",
    ),
    AssistantMessage(ParallelFunctionCall([plus_1_2, minus_2_1])),
    FunctionResultMessage(3, plus_1_2),
    FunctionResultMessage(1, minus_2_1),
    UserMessage("Now add 4 to both results."),
    functions=[plus, minus],
    model=chat_model,
)
def do_math() -> ParallelFunctionCall[int]: ...


output = do_math()
print(list(output))
# > [FunctionCall(<function plus at 0x10c3584c0>, 3, 4),
#    FunctionCall(<function plus at 0x10c3584c0>, 1, 4)]
output()
# (7, 5)

13:26:17.151 Calling chatprompt-function do_math
13:26:17.158   Chat Completion with 'gpt-4o' [LLM]
13:26:18.949 streaming response from 'gpt-4o' took 0.01s [LLM]
[FunctionCall(<function plus at 0x110c40680>, 3, 4), FunctionCall(<function plus at 0x110c40680>, 1, 4)]
13:26:18.950 Executing parallel function call
13:26:18.951   Executing function call plus
13:26:18.951   Executing function call plus


(7, 5)